# Notebook 4 – Next word steering

***

In this notebook we will showcase the effect of steering on the next word predicted using causal language models (generative LLMs trained to predict the next tokens given a sequence of previous tokens).

The notebook includes these :

1. [Displaying the next token from a prompt](#display-top-next-token---based-on-a-prompt)
2. [Steering the prompt using a steering vector, displaying top tokens now](#display-top-next-tokens---after-steering)
3. [Generating longer text based on a prompt and steering](#generate-longer-strings-of-text)

In [2]:
# Append the path to the Functions directory

import sys
sys.path.append('../Functions')
sys.path.append('../Features')

## Functions used in this Notebook:

### From "Next_word_steering.py":
- [initialize_model_and_tokenizer](#importing-python-functions-and-data) - Set the model and tokenizer
- [display_next_tokens](#display-top-next-token---based-on-a-prompt) - Print next token predictions
- [get_embedding_gpt](#create-steering-vector---using-own-sentences) - Embed a list of strings (create a steering vector)
- [get_steering_vector_gpt](#create-steering-vector---using-a-feature-file) - Create steering vector based on a feature file
- [display_steered_next_tokens](#display-top-next-tokens---after-steering) - Print next token predictions after steering
- [generate_steered_text](#generate-longer-strings-of-text) - Generate longer strings of text based on predictions and steering


***

## Import python functions and set model

In [3]:
from Next_word_steering import display_next_tokens, display_steered_next_tokens, get_embedding_gpt, get_steering_vector_gpt, initialize_model_and_tokenizer, generate_steered_text, generate_text

In [4]:
model_name = "openai-community/gpt2"

model, tokenizer = initialize_model_and_tokenizer(model_name)

/opt/anaconda3/envs/embed/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:820: UserWarning: `return_dict_in_generate` is NOT set to `True`, but `output_hidden_states` is. When `return_dict_in_generate` is not `True`, `output_hidden_states` is ignored.
  warnings.warn(


***

## Display top next token - based on a prompt

Input any sentence as the prompt

In [5]:
prompt = "I am a woman, my doctor is a"

display_next_tokens(model, tokenizer, prompt)


Prompt: 'I am a woman, my doctor is a'

Top 10 next-token predictions:

' man': 0.4267
' woman': 0.3539
' doctor': 0.0781
' physician': 0.0069
' feminist': 0.0069
' male': 0.0062
' lady': 0.0052
' Muslim': 0.0031
' gentleman': 0.0030
' girl': 0.0024


## Create steering vector - using own sentences

`get_embedding_gpt` creates an embedding of a list of strings, in the layer wanted.

- `normalize (True/False)` normalizes the embedding

In [9]:
layer_to_steer = 10
steering_coefficient = 50

# Example of strings: Women
steering_sentences = [
    "She braided her daughter’s hair with one hand while sending an email with the other — and no one questioned it.",
    "The midwife stood calm as storms, her voice steadier than the monitors beeping beside her.",
    "Wearing heels or combat boots, she walks like the world owes her space — and it does.",
    "She bleeds monthly and still runs marathons, meetings, and entire households.",
    "The senator adjusted her blazer and silenced the room before saying a single word.",
    "She is the matriarch, the memory-keeper, the one everyone calls when things fall apart.",
    "Her lipstick is warpaint, and her silence is strategy.",
    "From nursery rhymes to protest chants, her voice has always carried more than melody.",
    "She stitched every family story into the quilt that now warms three generations.",
    "She grew life inside her, lost sleep for years, and still built a business from scratch.",
    "The grandmother who crossed borders with babies strapped to her chest — that’s who she is.",
    "She is the girl told to smile, the teen told to shrink, the woman who refused.",
    "Behind every medal, there's a ponytail soaked in sweat and defiance.",
    "She signs her name where others once wrote hers for her.",
    "You can find her in every history book margin — not because she wasn’t there, but because someone tried to erase her."]

steering_vector = get_embedding_gpt(model, tokenizer, steering_sentences, layer_to_steer, normalize=True)

## Create steering vector - using a feature file

`get_steering_vector_gpt` creates a steering vector using the feature wanted and the layer wanted. This function uses the function `get_embedding_gpt` to create the embedding vector of the text.

This function uses the function `import_feature_texts(f"Features/{feature}")`, and reqires the user to have a folder called "Features" with a collection of feature texts inside files called "feature.txt" and optionally "opposite.txt".

- `normalize` normalizes the steering vector (but the get_embedding function is never normalized at the same time)

In [6]:
# Uncomment this cell to use the steering vector from your feature file
'''
layer_to_steer = 11
steering_coefficient = 2
feature = "Love"

steering_vector = get_steering_vector_gpt(model, tokenizer, feature, layer_to_steer, normalize=True)
'''

'\nlayer_to_steer = 11\nsteering_coefficient = 2\nfeature = "Love"\n\nsteering_vector = get_steering_vector_gpt(model, tokenizer, feature, layer_to_steer, normalize=True)\n'

## Display top next tokens - after steering

`display_steered_next_tokens` steers the prompt using the steering vector, and displays the top "k" predictions for the next token.

In [10]:
display_steered_next_tokens(model, tokenizer, prompt, layer_to_steer, steering_vector, steering_coefficient, k=10)


Prompt: 'I am a woman, my doctor is a'
Steering coefficient: 50

Top 10 next-token predictions:

' woman': 0.4444
' man': 0.3300
' doctor': 0.0572
' lady': 0.0081
' male': 0.0077
' feminist': 0.0076
' physician': 0.0051
' girl': 0.0043
' female': 0.0034
' gentleman': 0.0030


***

## Generate longer strings of text

Insert a new prompt and select the layer, vector and use the function to generate a longer sequence of tokens before steering.

`generate_text` generates a longer sentence or text using the predictions for the next tokens

- `max_tokens (int)` is the maximum amount of tokens that can be generated

- `stop_token="."` can make the generation of tokens stop at a "." thus creating one sentence, if `stop_token=None`, it stops only when "max_tokens" is reached

- `temperature (float)` controls randomness: lower = more deterministic, higher = more creative
    - **1.0** = Baseline
    - **<1.0** = Less random, sharpens distribution, picking more probable words
    - **>1.0** = More creative and diverse output, picks less probable words
    - **0** = Always picks most probable token

In [36]:
new_prompt = "In the year 3000, humanity"

generate_text(model, tokenizer, new_prompt, stop_token=".", max_tokens=20, temperature=0)

'In the year 3000, humanity has been on a collision course with the sun.'

Now, select the layer, feature, steering coefficient and normalization desired, and create the steering vector.

In [27]:
layer_to_steer = 11
feature = "War"
steering_coefficient = 50
normalize = True

steering_vector = get_steering_vector_gpt(model, tokenizer, feature, layer_to_steer, normalize=normalize)

`generate_steered_text` **steers the prompt** using the steering vector and then generates a longer sentence or text using the predictions for the next tokens

The parameters are the same as for `generate_text`, but also includes the `layer_to_steer`, `steering_vector` and `steering_coefficient`

In [34]:
generate_steered_text(model, tokenizer, new_prompt, layer_to_steer, steering_vector, steering_coefficient, stop_token=".", max_tokens=20, temperature=1)

'In the year 3000, humanity prospered not only by men claiming power in a clear vacuum, but also by what took them'

## Checkpoint
This cell will verify that all next-word steering functionality has been executed successfully

In [38]:
# 🎯 CHECKPOINT: Next-Word Steering Verification
print("="*60)
print("📋 NEXT-WORD STEERING CHECKPOINT")
print("="*60)

try:
    # Verify model and tokenizer initialization
    if 'model' in locals() and 'tokenizer' in locals():
        print(f"✅ Model initialized: {model_name}")
        print(f"✅ Tokenizer initialized")
    else:
        raise Exception("Model or tokenizer not initialized")
    
    # Verify steering parameters
    print(f"✅ Layer used for steering: {layer_to_steer}")
    print(f"✅ Steering coefficient: {steering_coefficient}")
    
    # Verify steering vector creation
    if 'steering_vector' in locals() and steering_vector is not None:
        import torch
        print(f"✅ Steering vector created: {steering_vector.shape}")
        print(f"✅ Steering vector norm: {torch.norm(steering_vector):.4f}")
        
        # Check if steering_sentences or feature was used
        if 'steering_sentences' in locals():
            print(f"✅ Steering method: Custom sentences ({len(steering_sentences)} sentences)")
        elif 'feature' in locals():
            print(f"✅ Steering method: Feature file ('{feature}')")
    else:
        print("❌ Warning: No steering vector found")
    
    # Verify prompts
    if 'prompt' in locals():
        print(f"✅ Base prompt: '{prompt}'")
    
    if 'new_prompt' in locals():
        print(f"✅ Text generation prompt: '{new_prompt}'")
        
    # Check if functions were executed (by checking for their outputs)
    # Note: This is a basic check and assumes the functions were called if variables are defined
    
    print("\n🔍 Function Execution Status:")
    if 'display_next_tokens' in locals():
        print("✅ display_next_tokens - Next token predictions viewed")
    
    if 'display_steered_next_tokens' in locals():
        print("✅ display_steered_next_tokens - Steered next token predictions viewed")
    if 'generate_text' in locals():
        print("✅ generate_text - Text generation completed")
    if 'generate_steered_text' in locals():
        print("✅ generate_steered_text - Steered text generation completed")
    
    print("\n💡 Next Steps:")
    print("1. Try different prompts to observe how steering affects predictions")
    print("2. Experiment with different steering coefficients")
    print("3. Use different features or custom steering sentences")
    print("4. Modify the layer being steered to observe different effects")
    
    print("="*60)
    print("🎯 CHECKPOINT PASSED - Next-word steering functioning correctly!")
    print("="*60)

except Exception as e:
    print("❌ CHECKPOINT FAILED")
    print(f"💥 Error: {str(e)}")
    print("🔧 Please check previous cells and ensure model, tokenizer and steering vector were initialized")
    print("💡 Tip: Make sure to run the initialization cells before trying to use the steering functions")

📋 NEXT-WORD STEERING CHECKPOINT
✅ Model initialized: openai-community/gpt2
✅ Tokenizer initialized
✅ Layer used for steering: 11
✅ Steering coefficient: 50
✅ Steering vector created: torch.Size([768])
✅ Steering vector norm: 1.0000
✅ Steering method: Custom sentences (15 sentences)
✅ Base prompt: 'I am a woman, my doctor is a'
✅ Text generation prompt: 'In the year 3000, humanity'

🔍 Function Execution Status:
✅ display_next_tokens - Next token predictions viewed
✅ display_steered_next_tokens - Steered next token predictions viewed
✅ generate_text - Text generation completed
✅ generate_steered_text - Steered text generation completed

💡 Next Steps:
1. Try different prompts to observe how steering affects predictions
2. Experiment with different steering coefficients
3. Use different features or custom steering sentences
4. Modify the layer being steered to observe different effects
🎯 CHECKPOINT PASSED - Next-word steering functioning correctly!
